In [8]:
from aicspylibczi import CziFile
import numpy as np
from numpy import random as rd
import tifffile as tf
from pathlib import Path

from matplotlib import pyplot as plt

import re
from pathlib import Path
import shutil
from os import remove, rmdir
from sys import argv

In [9]:
# helper functions

def isempty(dir_path):
    # Checks a directory to see if it contains any files
    
    path = Path(dir_path)
    has_next = next(path.iterdir(), None)
    if has_next is None:
        return True
    return False

def create_dir(dir_path):
    # Create an empty directory at dir_path if it doesn't yet exist
    try:
        Path.mkdir(dir_path)
    except FileExistsError:
        if not isempty(input_dir):
            raise Exception('Directory ' +dir_path.name + ' is not empty.')

In [10]:
rootdir = "/media/austin/DrosophilaMelanogaster/IPI/8plex/data/"
savedir = "/media/austin/DrosophilaMelanogaster/IPI/8plex/"
files = ["IPICRC055_8plex.czi"]

In [11]:
#channels = ['DAPI','FoxP3','CD4','CD45','CD8']
channels = ['DAPI','HLADR','CD8','CD163','CD4','XCR1','CD3','PDL1','EPCAM']
#channels=['Brightfield']

is_bf = False
tile_size = 2048

for f in files:
    
    f = Path(rootdir, f)
    czi = CziFile(f)
    im_shape = czi.get_dims_shape()
    
    if not is_bf:
        
        nchannels = im_shape[0]['C'][1]
        assert(len(channels)==nchannels)
    elif is_bf:
        
        nchannels = 1

    expdir = Path(rootdir, f.stem)
    try:
        Path.mkdir(expdir)
    except FileExistsError: pass

    print("Reading " + f.name)

    for c in np.arange(nchannels):
        
        fov = 0
        print("Reading " + channels[c] + " channel")    
        im = czi.read_mosaic(C=c)
        
        if is_bf:
            _,w,h,rgb = im.shape
        else:
            _,w,h = im.shape

        rows = list(np.arange(0,w,tile_size))
        cols = list(np.arange(0,h,tile_size))
        

        for x in rows:

            if w-x < tile_size:
                x_end = w
            else:
                x_end = x+tile_size

            for y in cols:
                
                
                # create empty tile, useful for padding out incomplete tiles at the edges with zeros

                if is_bf:
                    tile = np.zeros((tile_size, tile_size, 3))
                else:
                    tile = np.zeros((tile_size, tile_size))

                if h-y < tile_size:
                    y_end = h
                else:
                    y_end = y+tile_size
                    
                if is_bf:
                    tile[0:x_end-x, 0:y_end-y,:] = im[0, x:x_end, y:y_end,:]
                else:
                    tile[0:x_end-x, 0:y_end-y] = im[0, x:x_end, y:y_end]


                savedir = Path(expdir, 'fov' + str(fov))

                try:
                    Path.mkdir(savedir)
                except FileExistsError:
                    pass
                
                tf.imsave(str(Path(savedir, channels[c] + ".tiff")), tile)

                if c==0:
                    with open(Path(expdir, "tile_metadata.txt"), "a") as f:
                        f.write(",".join(["fov" + str(fov), str(x), str(x_end), str(y), str(y_end)]) + "\n")

                fov += 1
                
    #for exp in exp_dirs:

    fov_dirs = [fov for fov in Path(expdir).iterdir() if fov.is_dir()]

    #Create required directory structure for DeepCell
    input_dir = Path(expdir,"input_data")
    create_dir(input_dir)
    single_tiff_dir = Path(input_dir, "single_channel_inputs")
    create_dir(single_tiff_dir)
    mibitiff_dir = Path(input_dir, "mibitiff_inputs")
    create_dir(mibitiff_dir)
    deepcell_input_dir = Path(input_dir,"deepcell_input")
    create_dir(deepcell_input_dir)
    deepcell_output_dir = Path(expdir, "deepcell_output")
    create_dir(deepcell_output_dir)

    for f in fov_dirs:

        res = re.match("fov\d+",f.name)[0]
        create_dir(Path(single_tiff_dir,res))

        tiff_dir = Path(single_tiff_dir,res,"TIFs")
        print(tiff_dir)
        create_dir(tiff_dir)

        tiffs = [im_file for im_file in Path(f).iterdir() if (im_file.name.endswith(".tif") or im_file.name.endswith(".tiff"))]

        for tf in tiffs:
            shutil.move(str(tf), str(Path(tiff_dir, tf.name)))

        rmdir(f)

Reading IPICRC055_8plex.czi
Reading DAPI channel
Reading HLADR channel
Reading CD8 channel
Reading CD163 channel
Reading CD4 channel
Reading XCR1 channel
Reading CD3 channel
Reading PDL1 channel
Reading EPCAM channel
/media/austin/DrosophilaMelanogaster/IPI/8plex/data/IPICRC055_8plex/input_data/single_channel_inputs/fov0/TIFs
/media/austin/DrosophilaMelanogaster/IPI/8plex/data/IPICRC055_8plex/input_data/single_channel_inputs/fov1/TIFs
/media/austin/DrosophilaMelanogaster/IPI/8plex/data/IPICRC055_8plex/input_data/single_channel_inputs/fov10/TIFs
/media/austin/DrosophilaMelanogaster/IPI/8plex/data/IPICRC055_8plex/input_data/single_channel_inputs/fov11/TIFs
/media/austin/DrosophilaMelanogaster/IPI/8plex/data/IPICRC055_8plex/input_data/single_channel_inputs/fov12/TIFs
/media/austin/DrosophilaMelanogaster/IPI/8plex/data/IPICRC055_8plex/input_data/single_channel_inputs/fov13/TIFs
/media/austin/DrosophilaMelanogaster/IPI/8plex/data/IPICRC055_8plex/input_data/single_channel_inputs/fov14/TIFs
/